In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time

In [2]:
def get_data():
    html_doc = requests.get('https://www.moneycontrol.com/stocks/marketstats/indexcomp.php?optex=NSE&opttopic=indexcomp&index=9')
    soup = BeautifulSoup(html_doc.text, 'html.parser')
    rows = soup.findAll('table')[1].findAll('tr')[1:]
    data = []
    for row in rows:
        cols = row.findAll('td')
        temp = []
        for i, col in enumerate(cols):
            if i == 0:
                temp.append(col.find('b').text)
            else:
                temp.append(col.text)
        data.append(temp)
    return pd.DataFrame(data, columns=['Company Name', 'Industry', 'Last Price', 'Change', '%Chg', 'Mkt Cap'])

In [3]:
def alert(company_name, percent_change):
    print(f"{company_name}: {percent_change}")
    
def check_for_alert(prev_change, new_change):
    res = []
    l = len(prev_change)
    for i in range(l):
        if abs(float(new_change[i])-float(prev_change[i])) >= 2:
            res.append(i)
    return res

In [ ]:
start_time = time.time()
df = get_data()
running_time = 0
data = [df]
total_time = 20 #minutes
while running_time <= total_time*60:
    time.sleep(30)
    df = get_data()
    data.append(df)
    if len(data) % 4 == 0:
        temp = check_for_alert(data[0][:]['%Chg'].values, data[-1][:]['%Chg'].values)
        for i in temp:
            alert(data[-1][i]['Company Name'], abs(float(data[-1][i]['%Chg'])-float(data[0][i]['%Chg'])))
        data = data[1:]
    running_time = time.time()-start_time